### MANDATORY ASSIGNMENT 2

In [1]:
from sklearn import datasets

In [2]:
import numpy as np

In [3]:
iris = datasets.load_iris()

In [4]:
X = iris.data
Y = iris.target

#### Task 1) data exploration

In [5]:
len(X)

150

In [6]:
print(X.shape, Y.shape)

(150, 4) (150,)


In [7]:
print(np.min(X), np.max(X))
print(np.min(Y), np.max(Y))

0.1 7.9
0 2


##### 2) Encoding and pre-processing

In [93]:
#at first we normalize the data from 0 to pi, and then implement angle encoding
from sklearn.preprocessing import MinMaxScaler
from qiskit import QuantumCircuit, transpile, assemble
from qiskit_aer import Aer, AerSimulator
from qiskit.visualization import plot_histogram
from qiskit.circuit import Parameter
from qiskit_algorithms.optimizers import SPSA
import random

In [9]:
scaler = MinMaxScaler(feature_range=(0, np.pi))
X = scaler.fit_transform(X)

In [10]:
def angle_encoding(qc, sample):
    for qubit in range(len(qc.qubits)):
        qc.rx(sample[qubit], qubit)

    

##### 3) choosing Loss function

In [135]:
#Implementing cross-entropy loss
from sklearn.metrics import log_loss

##### 4) splitting data

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
X_train, X_temp, y_train, y_temp = train_test_split(X, Y, test_size=0.3, random_state=42) # 70% training 
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42) # 15% validation, 15% testing

In [14]:
num_qubits = 4
num_layers = 3

In [15]:
int('1111', 2)

15

In [16]:
theta = '\u03B8'

def real_amplitudes(data_point,parameters, layers = num_layers):
    qc = QuantumCircuit(num_qubits)
    angle_encoding(qc, data_point)

    param_index = 0

    for layer in range(layers):
        for qubit in range(len(qc.qubits)):
            qc.ry(parameters[param_index], qubit)
            param_index += 1
        qc.barrier()
        
        for qubit in range(len(qc.qubits)-1):
            qc.cx(qubit, qubit+1)
        qc.barrier()

    return qc
    

In [17]:
def data_decoding(output):
    return int(output, 2) % 3 

In [156]:
def objective_function(updated_params):
    loss = 0
    transpilation_cache = {}
    backend = AerSimulator(method = 'statevector')
    predicted_probabilites = []
    for x, y in zip(X_train, y_train):
        qc = real_amplitudes(x, updated_params, layers=num_layers)
        qc.measure_all()
        shots = 100

        # If circuit has already been transpiled, use the stored version. This reduces runtime
        circuit_key = tuple(x)  # Convert input to immutable tuple for dict key
        if circuit_key not in transpilation_cache:
            transpiled_qc = transpile(qc, backend)  # Done only for new circuits
            transpilation_cache[circuit_key] = transpiled_qc

        tqc = transpilation_cache[circuit_key]
        
        job = backend.run(tqc, shots=shots)
        result = job.result()
        counts = result.get_counts(qc)

        count_classes = {0: 0, 1: 0, 2: 0}
        for output, count in counts.items():
            class_num = data_decoding(output)
            count_classes[class_num] += count / shots
        

        predicted_probabilites.append([count_classes[0], count_classes[1], count_classes[2]])        
        
     
    logloss = log_loss(y_train, predicted_probabilites)

    print(f"Parameters: {updated_params} loss: {logloss}")
    return logloss
    

In [166]:
rng = np.random.default_rng(42)
initial_parameters = rng.uniform(0, np.pi, num_qubits * num_layers)

# Gradient Descent optimizer
optimizer = SPSA(maxiter=50)

# Optimize the parameters
optimized = optimizer.minimize(fun=objective_function, x0=initial_parameters)

print("Optimized Parameters:", optimized.x)
print("Minimum Loss:", optimized.fun)


Parameters: [2.23145464 1.17877728 2.89736492 2.39084628 0.09586686 2.86500801
 2.5911909  2.66949385 0.60248085 1.21492915 1.36489635 2.71151808] loss: 0.9964246761938709
Parameters: [2.63145464 1.57877728 2.49736492 1.99084628 0.49586686 3.26500801
 2.1911909  2.26949385 0.20248085 1.61492915 0.96489635 3.11151808] loss: 1.1868912373519924
Parameters: [2.23145464 1.17877728 2.49736492 1.99084628 0.09586686 3.26500801
 2.5911909  2.66949385 0.20248085 1.61492915 0.96489635 3.11151808] loss: 1.3269231106091475
Parameters: [2.63145464 1.57877728 2.89736492 2.39084628 0.49586686 2.86500801
 2.1911909  2.26949385 0.60248085 1.21492915 1.36489635 2.71151808] loss: 1.0202764107643751
Parameters: [2.23145464 1.17877728 2.89736492 1.99084628 0.09586686 3.26500801
 2.1911909  2.66949385 0.20248085 1.21492915 1.36489635 3.11151808] loss: 1.2142678803398756
Parameters: [2.63145464 1.57877728 2.49736492 2.39084628 0.49586686 2.86500801
 2.5911909  2.26949385 0.60248085 1.61492915 0.96489635 2.711

In [167]:
def predict(data_point, optimized_params):
    qc = real_amplitudes(data_point, optimized_params)
    qc.measure_all()

    backend = AerSimulator(method = 'statevector')
    tqc = transpile(qc, backend)
    shots = 100
    job = backend.run(tqc, shots=shots)
    result = job.result()
    counts = result.get_counts(qc)

    count_classes = {0: 0, 1: 0, 2: 0}
        
    # Decode each measurement outcome and aggregate counts for each class
    for output, count in counts.items():
        class_num = data_decoding(output)
        count_classes[class_num] += count
    
    # Calculate probabilities for each class
    probabilities = {class_num: count / shots for class_num, count in count_classes.items()}
    
    # Determine the predicted class by choosing the class with the highest probability
    predicted_class = max(probabilities, key=probabilities.get)
    
    return predicted_class

In [168]:
def predict_dataset(X, optimized_params):
    return [predict(data_point, optimized_params) for data_point in X]

In [169]:
predictions = predict_dataset(X_test, optimized.x)
predictions

[0, 2, 2, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 2, 1, 0, 1, 1, 1, 2, 0, 0, 1]

In [170]:
from sklearn.metrics import accuracy_score

In [171]:
score = accuracy_score(y_test, predictions)
score

0.8695652173913043

In [172]:
y_test

array([0, 2, 2, 0, 2, 1, 1, 0, 1, 1, 1, 2, 1, 2, 1, 0, 1, 1, 1, 2, 0, 0,
       2])